In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import sklearn
import tqdm.notebook as tqdm

### Load the Dataset

In [2]:
!ls dataset/

 dataset.csv  'Variable Names (1).xlsx'


In [30]:
df = pd.read_csv("dataset/dataset.csv", index_col = [0])
df


,ID,CTFLAG,ANYFX,FRAX_SCORE,PARKINS,RHEUMAT,OSTEOPOR,ARTHRIT,CANC_F30,CATARACT,...,F60VITA,TEXPWK,WALKSPD,BKBONE,BKHIP,BKBACK,BKLARM,SMOKING,YEARS_MENOPAUSE,DUR_MENA_MENO
0,131073,1,0,6.14,0.0,0.0,0.0,0.0,0.0,0.0,...,975.84083,2.50000,3.0,1.0,0.0,0.0,1.0,1.0,10.0,-45.0
1,262147,1,0,8.05,0.0,8.0,0.0,1.0,0.0,0.0,...,848.40762,26.83333,3.0,0.0,0.0,0.0,0.0,1.0,13.0,-44.0
2,131075,0,0,12.88,0.0,8.0,0.0,1.0,0.0,1.0,...,629.72861,21.00000,3.0,1.0,0.0,0.0,0.0,1.0,11.0,-45.0
3,262149,0,0,8.78,0.0,8.0,0.0,1.0,0.0,0.0,...,339.14853,32.83333,4.0,0.0,0.0,0.0,0.0,1.0,15.0,-45.0
4,262150,1,1,1.73,0.0,0.0,0.0,0.0,0.0,0.0,...,1574.51101,21.83333,3.0,0.0,0.0,0.0,0.0,0.0,19.0,-30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74769,262130,1,0,3.07,0.0,0.0,0.0,0.0,0.0,0.0,...,668.50414,0.00000,3.0,1.0,0.0,0.0,0.0,0.0,2.0,-46.0
74770,131066,1,0,3.94,0.0,0.0,0.0,0.0,0.0,0.0,...,334.67271,7.50000,3.0,0.0,0.0,0.0,0.0,0.0,7.0,-46.0
74771,262131,0,0,4.45,0.0,0.0,0.0,0.0,0.0,0.0,...,1195.77043,17.08333,3.0,0.0,0.0,0.0,0.0,1.0,1.0,-47.0
74772,131068,1,0,8.54,0.0,0.0,0.0,0.0,0.0,0.0,...,1169.27512,0.00000,9.0,1.0,0.0,0.0,0.0,0.0,13.0,-45.0


In [53]:
#extract cohort, labels, patid
df = df[df["CTFLAG"] == 1]

#do not drop frax score yet
labels = df["ANYFX"]
dataset = df.drop(columns = ["ANYFX", "CTFLAG", "ID"]) 

print("Dataset Shape:", df.shape)

Dataset Shape: (31470, 67)


In [32]:
print(labels.value_counts())

ANYFX
0    27835
1     3635
Name: count, dtype: int64


### Training Loop

In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

def FRAX_maximize_youden_j(y_true: pd.Series, y_prob: pd.Series) -> float:
    """
    Finds the optimal threshold that maximizes Youden’s J Statistic (TPR - FPR).

    :param y_true: Pandas Series of true binary labels (0 or 1).
    :param y_prob: Pandas Series of predicted probabilities.
    :return: The optimal threshold for classification.
    """
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    
    # Compute Youden’s J statistic
    j_scores = tpr - fpr

    # Find the optimal threshold (maximum J score)
    best_threshold = thresholds[np.argmax(j_scores)]

    print(f"Optimal Threshold (Max Youden's J): {best_threshold:.4f}")

    return best_threshold



In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample

import pandas as pd
import numpy as np

def weighted_downsample(X: pd.DataFrame, 
                        y: pd.Series, 
                        feature_list: list = ["ETHNICNIH", "RACENIH"],
                        weights_dict: dict = None, 
                        frac: float = 0.5, 
                        random_state: int = 42):

    valid_features = [f for f in feature_list if f in X.columns]
    if not valid_features:
        raise ValueError("None of the specified features exist in X.")

    #no weights_dict is provided
    if weights_dict is None:
        weights_dict = {feature: 1 / len(valid_features) for feature in valid_features}

    total_weight = sum(weights_dict.values())
    normalized_weights = {k: v / total_weight for k, v in weights_dict.items() if k in valid_features}
    sampling_weights = X[valid_features].mul(normalized_weights).sum(axis=1)
    sampling_weights = np.maximum(sampling_weights, 1e-10)  # Avoid zero probability
    sampling_weights /= sampling_weights.sum()
    downsampled_indices = (
        pd.concat([X, y], axis=1) 
        .groupby(y.name, group_keys=False)  
        .apply(lambda group: group.sample(frac=frac, weights=sampling_weights.loc[group.index], random_state=random_state))
        .index
    )

    X_downsampled = X.loc[downsampled_indices]
    y_downsampled = y.loc[downsampled_indices]

    return X_downsampled, y_downsampled


def weighted_downsample_LABELS(df, labels, target_ratio=0.5, random_state=42):
    """
    Performs weighted downsampling to balance classes in the dataset.

    :param df: DataFrame with features.
    :param labels: Series with target labels.
    :param target_ratio: Desired ratio of the minority class.
    :param random_state: Random seed for reproducibility.
    :return: Downsampled features (X) and labels (y).
    """
    # Combine features and labels into one DataFrame
    df['label'] = labels
    
    # Identify majority and minority classes
    class_counts = df['label'].value_counts()
    min_class = class_counts.idxmin()
    maj_class = class_counts.idxmax()

    # Compute the target number of samples for the majority class
    n_min = class_counts[min_class]
    n_maj = int(n_min / target_ratio - n_min)

    # Downsample the majority class
    df_minority = df[df['label'] == min_class]
    df_majority_downsampled = resample(df[df['label'] == maj_class], 
                                       replace=False, 
                                       n_samples=n_maj, 
                                       random_state=random_state)

    # Combine the balanced dataset
    df_balanced = pd.concat([df_minority, df_majority_downsampled])

    # Separate features and labels again
    y_balanced = df_balanced['label']
    X_balanced = df_balanced.drop(columns=['label'])

    return X_balanced, y_balanced


In [48]:
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")

def evaluate_model(y_true, y_pred, y_prob=None, descr = None):
    """
    Computes and prints standard classification metrics: Accuracy, AUC, Precision, Recall, and F1-score.

    :param y_true: List or array of true labels (0 or 1).
    :param y_pred: List or array of predicted labels (0 or 1).
    :param y_prob: List or array of predicted probabilities (optional, needed for AUC).
    :return: Dictionary containing Accuracy, AUC, Precision, Recall, and F1-score.
    """
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1-score": f1_score(y_true, y_pred, zero_division=0),
        "AUC": roc_auc_score(y_true, y_prob) 
    }

    # Print metrics
    if descr:
        print(descr)
    for key, value in metrics.items():
        print(f"\t{key}: {value:.4f}" if value is not None else f"{key}: N/A (Only one class present)")

    return metrics

    
    #extract featuresd
    y_pred = model.predict(X_train_balanced)
    y_prob = model.predict_proba(X_train_balanced)[:, 1]

def eval_run(model, x, y, descr = None):
    y_pred = model.predict(x)
    y_prob = model.predict_proba(x)[:, 1]
    evaluate_model(y, y_pred, y_prob, descr = descr)

def eval_frax(frax_scores, labels, descr = None, threshold = None):
    if threshold:
        pass
    else:
        threshold = FRAX_maximize_youden_j(labels, frax_scores)
    y_pred = (frax_scores >= threshold).astype(int)
    evaluate_model(labels, y_pred, frax_scores, descr = descr)
    return threshold



In [49]:
random_cv_avail = True

import scipy

param_dist = {
    'n_estimators': scipy.stats.randint(100, 500),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': scipy.stats.randint(2, 20),
    'min_samples_leaf': scipy.stats.randint(1, 10),
    'max_features': ['sqrt', 'log2']
}

In [52]:
from sklearn.model_selection import RandomizedSearchCV

n_splits = 5
random_state = 45
#downsample
target_ratio=0.5

skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
scores = []

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42)

for fold, (train_idx, test_idx) in enumerate(skf.split(dataset, labels)):
    print(f"Fold {fold+1}/{n_splits}--------------------------")

    # Split train and test sets
    X_train, X_test = dataset.iloc[train_idx], dataset.iloc[test_idx]
    y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]
    
    # Perform weighted downsampling on training data
    X_train_balanced, y_train_balanced = weighted_downsample_LABELS(X_train, y_train, target_ratio = 0.3)
    print("Training Fold Label Distr:", dict(pd.Series(y_train_balanced).value_counts()))

    #get frax after downsample
    frax_score_train, frax_score_test = X_train_balanced["FRAX_SCORE"], X_test["FRAX_SCORE"]
    X_train_balanced, X_test = X_train_balanced.drop(columns = ["FRAX_SCORE"]), X_test.drop(columns = ["FRAX_SCORE"])

    if random_cv_avail == True:
        random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=20, cv=5, scoring='accuracy', n_jobs=-1, random_state=42)
        random_search.fit(X_train_balanced, y_train_balanced)
        print("Best Parameters:", random_search.best_params_)
        model = random_search.best_estimator_

    eval_run(model, X_train_balanced, y_train_balanced, descr = "MODEL Train:)
    FRAX_threshold = eval_frax(frax_score_train, y_train_balanced, descr = "FRAX Train:")
    
    # Evaluate on the original test set (without downsampling)
    eval_run(model, X_test, y_test, descr = "MODEL Test:")
    eval_frax(frax_score_test, y_test, descr = "FRAX Test:", threshold = FRAX_threshold)

    print("Test Fold Label Distr:", dict(pd.Series(y_test).value_counts()))
    print("\n")


Fold 1/5--------------------------
Training Fold Label Distr: {0: 6785, 1: 2908}
Best Parameters: {'max_depth': 30, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 7, 'n_estimators': 153}
	Accuracy: 0.9782
	Precision: 1.0000
	Recall: 0.9274
	F1-score: 0.9624
	AUC: 1.0000
Optimal Threshold (Max Youden's J): 6.9800
FRAX Train Metrics:
	Accuracy: 0.5927
	Precision: 0.3637
	Recall: 0.4773
	F1-score: 0.4128
	AUC: 0.5836
Test Split:
	Accuracy: 0.8834
	Precision: 0.3704
	Recall: 0.0138
	F1-score: 0.0265
	AUC: 0.5878
FRAX Test Metrics:
	Accuracy: 0.6214
	Precision: 0.1409
	Recall: 0.4470
	F1-score: 0.2143
	AUC: 0.5695
Test Fold Label Distr: {0: 5567, 1: 727}


Fold 2/5--------------------------
Training Fold Label Distr: {0: 6785, 1: 2908}
Best Parameters: {'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 8, 'min_samples_split': 17, 'n_estimators': 336}
	Accuracy: 0.7531
	Precision: 1.0000
	Recall: 0.1771
	F1-score: 0.3009
	AUC: 0.9951
Optimal Threshold (Max You